#  Análisis y comparación de modelos SVR para predicción de precios de Airbnb en Madrid

## Info del dataset

In [17]:
# Cargo los datos del csv de data/
import pandas as pd
df = pd.read_csv('../data/airbnb.csv')
df

,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Centro,Justicia,40.424715,-3.698638,Entire home/apt,49,28,35,0.42,1,99
1,Centro,Embajadores,40.413418,-3.706838,Entire home/apt,80,5,18,0.30,1,188
2,Moncloa - Aravaca,Argüelles,40.424920,-3.713446,Entire home/apt,40,2,21,0.25,9,195
3,Moncloa - Aravaca,Casa de Campo,40.431027,-3.724586,Entire home/apt,55,2,3,0.13,9,334
4,Latina,Cármenes,40.403410,-3.740842,Private room,16,2,23,0.76,2,250
...,...,...,...,...,...,...,...,...,...,...,...
13316,Centro,Justicia,40.427500,-3.698354,Private room,14,1,0,0.00,1,10
13317,Chamberí,Gaztambide,40.431187,-3.711909,Entire home/apt,47,1,0,0.00,7,354
13318,Centro,Palacio,40.413552,-3.711461,Entire home/apt,60,2,0,0.00,1,17
13319,Centro,Universidad,40.425400,-3.709921,Entire home/apt,150,5,0,0.00,1,15


## LinearSVR vs SVR(rbf) - Template de modelado y evaluación

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.svm import LinearSVR, SVR

# =========== CARGA DE DATOS ============
df = pd.read_csv('../data/airbnb.csv')
print(f"Filas iniciales: {len(df)}")

#=========== SAMPLE DE LOS DATOS ============
# Muestreo para acelerar el proceso (opcional)
df = df.sample(n=5000, random_state=0)
print(f"Filas después del muestreo: {len(df)}")
print(f"Columnas despues del sample: {df.columns.tolist()}")

# ============ PREPROCESAMIENTO ============

# 1. Eliminar duplicados
df = df.drop_duplicates()
print(f"Filas después de eliminar duplicados: {len(df)}")

# 2. Eliminar outliers en price, minimum_nights y calculated_host_listings_count
cols_outliers = ['price', 'minimum_nights', 'calculated_host_listings_count']

# 2.1. Calcular Q1, Q3 e IQR por columna
Q1 = df[cols_outliers].quantile(0.25)
Q3 = df[cols_outliers].quantile(0.75)
IQR = Q3 - Q1

# 2.3 Crear y combinar máscaras (AND) para que la fila esté dentro de los límites en todas las columnas
masks = [(df[c] >= (Q1[c] - 1.5 * IQR[c])) & (df[c] <= (Q3[c] + 1.5 * IQR[c])) for c in cols_outliers]
mask = np.logical_and.reduce(masks)

# 2.4 Aplicar filtro y mostrar conteo antes/después
before = len(df)
df = df[mask].copy()
print(f"Filas antes: {before}, después de eliminar outliers: {len(df)}")

# 3. Eliminar columnas de latitud y longitud
df = df.drop(columns=['latitude', 'longitude'])
print(f"Columnas después de eliminar lat/long: {df.columns.tolist()}")

# 4. Eliminar la columna de neighbourhood si se desea (opcional)
df = df.drop(columns=['neighbourhood'])
print(f"Columnas después de eliminar neighbourhood: {df.columns.tolist()}")

# 5. Feature Engineering -POSTERIOR- (opcional)

# ============ PREPARACIÓN DE DATOS ============
X = df.drop(columns=['price'])

# Log-transform de la variable objetivo
y = np.log1p(df['price'].values)

# Columnas numéricas
num_cols = ['minimum_nights', 'number_of_reviews',
            'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
# Columnas categóricas
cat_cols = ['room_type', 'neighbourhood_group']

preproc = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
])

cv = KFold(n_splits=5, shuffle=True, random_state=0)

# ============ PIPE LinearSVR ============
pipe_linsvr = Pipeline([
    ('pre', preproc),
    ('model', LinearSVR(max_iter=50000, random_state=0))
])

param_grid_linsvr = {
    'model__C': np.logspace(-3, 3, 7)
}

gs_linsvr = GridSearchCV(pipe_linsvr, param_grid_linsvr, cv=cv,
                         scoring='neg_root_mean_squared_error', n_jobs=-1)
gs_linsvr.fit(X, y)
best_linsvr = gs_linsvr.best_estimator_

# ============ PIPE SVR (RBF) ============
pipe_svr = Pipeline([
    ('pre', preproc),
    ('model', SVR(kernel='rbf'))
])

param_grid_svr = {
    'model__C': [0.01, 0.1, 1, 10, 100],
    'model__gamma': ['scale', 'auto', 1e-2, 1e-3, 1e-4]
}

gs_svr = GridSearchCV(pipe_svr, param_grid_svr, cv=cv,
                      scoring='neg_root_mean_squared_error', n_jobs=-1)
gs_svr.fit(X, y)
best_svr = gs_svr.best_estimator_

# ============ EVALUACIÓN CV MANUAL ============
def cv_metrics(estimator, X, y_log, cv):
    rmses, maes, r2s = [], [], []
    for train_idx, val_idx in cv.split(X):
        est = estimator
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y_log[train_idx], y_log[val_idx]
        est.fit(X_tr, y_tr)
        y_pred_log = est.predict(X_val)
        y_pred = np.expm1(y_pred_log)
        y_true = np.expm1(y_val)
        # Calcula MSE y luego saca la raíz cuadrada para obtener RMSE
        rmses.append(np.sqrt(mean_squared_error(y_true, y_pred)))
        maes.append(mean_absolute_error(y_true, y_pred))
        r2s.append(r2_score(y_true, y_pred))
    return {'rmse': np.mean(rmses), 'mae': np.mean(maes), 'r2': np.mean(r2s)}

print('\n============ RESULTADOS ============')
print('LinearSVR CV:', cv_metrics(best_linsvr, X, y, cv))
print('SVR CV:', cv_metrics(best_svr, X, y, cv))
print(f'\nMejor C para LinearSVR: {gs_linsvr.best_params_}')
print(f'Mejores parámetros para SVR: {gs_svr.best_params_}')

Filas iniciales: 13321
Filas después del muestreo: 5000
Columnas despues del sample: ['neighbourhood_group', 'neighbourhood', 'latitude', 'longitude', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
Filas después de eliminar duplicados: 5000
Filas antes: 5000, después de eliminar outliers: 3904
Columnas después de eliminar lat/long: ['neighbourhood_group', 'neighbourhood', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']
Columnas después de eliminar neighbourhood: ['neighbourhood_group', 'room_type', 'price', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']

============ RESULTADOS ============
LinearSVR CV: {'rmse': np.float64(22.37201355271622), 'mae': np.float64(15.55219713628299), 'r2': np.float64(0.4636997813385304)}
SVR CV: {'rmse': np.float64(

## 📊 Preprocesamiento y datos

1. **Filas iniciales vs finales**:

   * Filas iniciales: 13,321 → Muestreo: 5,000 → Tras Outliers eliminados: 3,904

     > Ahora sí hay un recorte considerable (~22%), porque eliminaste outliers no solo en `price`, sino también en `minimum_nights` y `calculated_host_listings_count`. Esto reduce la varianza extrema y permite modelos más estables.

2. **Columnas eliminadas**:

   * Eliminación de lat/long y `neighbourhood`, reduciendo información geográfica precisa.

     > Esto simplifica el modelo, pero pierde detalle espacial; los modelos deben capturar patrones con `neighbourhood_group` en lugar de coordenadas precisas.

3. **Transformación del target**:

   * Log-transform de `price`, correcto para estabilizar la varianza y reducir la influencia de valores extremos.

---

## 🎯 Comparación de modelos

| Métrica | LinearSVR | SVR (RBF) | Diferencia |
| ------- | --------- | --------- | ---------- |
| RMSE    | 22.41     | 22.12     | -1.3%      |
| MAE     | 15.46     | 15.22     | -1.5%      |
| R²      | 0.464     | 0.478     | +3%        |

### Observaciones

1. **LinearSVR**:

   * R² = 0.464: Ahora el modelo lineal explica casi la mitad de la varianza, mucho mejor que en la versión anterior.

     > Esto se debe al filtrado más estricto de outliers, que reduce ruido y valores extremos que dificultaban el ajuste lineal.
   * Mejor C = 10: Requiere más flexibilidad (menos regularización) para ajustarse a la variabilidad de los datos.

2. **SVR (RBF)**:

   * R² = 0.478: Solo ligeramente mejor que LinearSVR, con mejoras mínimas en RMSE y MAE.

     > La ventaja del kernel no lineal disminuye, probablemente porque los outliers que generaban no linealidad extrema fueron eliminados.
   * Parámetros: C=10, gamma='0.01', lo que sugiere que un ajuste conservador es suficiente y no se requiere mucha complejidad para capturar la relación.

3. **Interpretación**:

   * El filtrado de outliers hace que la relación entre features y precio sea más lineal. Por eso, **LinearSVR casi iguala al SVR con RBF**.
   * El precio de Airbnb sigue mostrando variabilidad no capturada (R² < 0.5), lo que indica falta de features críticas, como amenities, estacionalidad o ubicación detallada.

---

## 💡 Recomendaciones

1. **Feature Engineering**:

   * Crear ratios e interacciones:

     * `reviews_per_month * number_of_reviews` → actividad del anfitrión.
     * `minimum_nights / availability_365` → ocupación relativa.
   * Incorporar variables de temporalidad (mes, temporada, fines de semana) si hay fecha de reserva.

2. **Ubicación**:

   * Mantener `neighbourhood` o coordenadas y aplicar clustering para crear variables geográficas derivadas.
   * Esto puede aumentar significativamente R² porque la ubicación es determinante en el precio.


3. **Validación**:

   * Estratificación por rangos de precio (low, mid, high) para evaluar desempeño en distintos segmentos.
   * Evaluar error relativo (%) además de absoluto para interpretar mejor RMSE y MAE en distintos rangos de precio.

---

### 🔍 Conclusión

* **Impacto del preprocesamiento**: La eliminación de outliers en varias columnas hizo que los datos fueran más lineales, reduciendo la ventaja del SVR con kernel RBF. Aunque aun así, el SVR con RBF es ligeramente mejor.
* **Rendimiento de modelos**: LinearSVR y SVR con RBF tienen rendimiento muy similar (R² ≈ 0.47), RMSE ≈ 22, MAE ≈ 15.
* **Próximo paso**: Mejorar la calidad de los features (ubicación precisa, amenities, temporalidad)

## SVR(rbf) - FineTuning + DataTreatment

In [48]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.base import BaseEstimator, TransformerMixin, clone
from haversine import haversine
import warnings
warnings.filterwarnings('ignore')

# ============ TRANSFORMADOR PERSONALIZADO PARA FEATURE ENGINEERING ============

class FeatureEngineer(BaseEstimator, TransformerMixin):
    """Crea features cuidando el data leakage y la multicolinealidad"""
    def __init__(self):
        self.sol_coords = (40.416775, -3.703790)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Distance to center
        X['distance_to_center'] = X.apply(
            lambda row: haversine(self.sol_coords, (row['latitude'], row['longitude'])),
            axis=1
        )

        # Eliminar lat/long después de crear distance_to_center
        X = X.drop(columns=['latitude', 'longitude'], errors='ignore')

        return X

# =========== CARGA Y PREPROCESAMIENTO ============
df = pd.read_csv('../data/airbnb.csv')
print(f"Filas iniciales: {len(df)}")

# Sample más grande para mejor generalización
df = df.sample(n=1000, random_state=0)
print(f"Filas después del muestreo: {len(df)}")

# ============ FILTRADO DE OUTLIERS MEJORADO ============
# Filtro más conservador para mantener más datos
q1 = df['price'].quantile(0.05)
q3 = df['price'].quantile(0.95)
iqr = q3 - q1
lower = q1 - 1.5 * iqr
upper = q3 + 1.5 * iqr
df = df[(df['price'] >= lower) & (df['price'] <= upper)].copy()

# Capear valores extremos
df['minimum_nights'] = df['minimum_nights'].clip(upper=365)

print(f"Filas después del filtrado: {len(df)}")

# Eliminar neighbourhood (alta cardinalidad y redundante con neighbourhood_group)
df = df.drop(columns=['neighbourhood'], errors='ignore')

# Reemplazar 0 por NaN para imputación
df['number_of_reviews'] = df['number_of_reviews'].replace(0, np.nan)
df['reviews_per_month'] = df['reviews_per_month'].replace(0, np.nan)
df['availability_365'] = df['availability_365'].replace(0, np.nan)

# ============ PREPARACIÓN DE DATOS ============ #
X = df.drop(columns=['price'])
y = np.log1p(df['price'].values)

# ============ PIPELINE COMPLETO SIN MULTICOLINEALIDAD ============

# Features numéricas SIN number_of_reviews (evitar correlación con review_intensity)
num_cols_base = ['minimum_nights', 'reviews_per_month','calculated_host_listings_count', 'availability_365']
num_cols_engineered = ['distance_to_center']
cat_cols = ['room_type', 'neighbourhood_group']

# Transformadores
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Median más robusto que mean
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('encoder', OneHotEncoder(
        handle_unknown='ignore',
        drop='first',           # Evitar multicolinealidad
        min_frequency=0.01      # Agrupar categorías raras (< 1%)
    ))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num_base', numeric_transformer, num_cols_base),
        ('num_eng', numeric_transformer, num_cols_engineered),
        ('cat', categorical_transformer, cat_cols)
    ],
    remainder='drop'
)

# Pipeline completo
pipeline = Pipeline([
    ('feature_eng', FeatureEngineer()),
    ('preprocessor', preprocessor),
    ('model', SVR(kernel='rbf', cache_size=6389))  # Valor de cache ajustado a memoria disponible
])

# ============ CROSS-VALIDATION ============
cv = KFold(n_splits=5, shuffle=True, random_state=0)

# Grid optimizado (menos combinaciones, rangos mejores)
param_grid = {
    'model__C': [0.1, 1, 5, 10, 50],
    'model__gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
    'model__epsilon': [0.01, 0.05, 0.1, 0.2]
}

# ============ GRID SEARCH ============
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=cv,
    scoring='neg_root_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    return_train_score=True  # Para detectar overfitting
)

print("\n Entrenando GridSearchCV...")
grid_search.fit(X, y)

print(f"\n Mejores parámetros: {grid_search.best_params_}")
print(f" Mejor score CV (neg_RMSE en log): {grid_search.best_score_:.4f}")

# ============ EVALUACIÓN EN ESCALA ORIGINAL ============
def evaluate_model_cv(pipeline, X, y, cv):
    """Evaluación correcta sin data leakage"""
    rmses, maes, r2s = [], [], []

    for fold, (train_idx, val_idx) in enumerate(cv.split(X), 1):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        model = clone(pipeline)
        model.fit(X_train, y_train)

        y_pred_log = model.predict(X_val)
        y_pred = np.expm1(y_pred_log)
        y_true = np.expm1(y_val)

        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)
        r2 = r2_score(y_true, y_pred)

        rmses.append(rmse)
        maes.append(mae)
        r2s.append(r2)

        print(f"  Fold {fold}: RMSE={rmse:.2f}, MAE={mae:.2f}, R²={r2:.4f}")

    return {
        'rmse_mean': np.mean(rmses),
        'rmse_std': np.std(rmses),
        'mae_mean': np.mean(maes),
        'mae_std': np.std(maes),
        'r2_mean': np.mean(r2s),
        'r2_std': np.std(r2s)
    }

print("\n Evaluando modelo en escala original...")
best_pipeline = grid_search.best_estimator_
metrics = evaluate_model_cv(best_pipeline, X, y, cv)

print("\n" + "="*50)
print("           RESULTADOS FINALES")
print("="*50)
print(f"RMSE: {metrics['rmse_mean']:.2f}€ ± {metrics['rmse_std']:.2f}€")
print(f"MAE:  {metrics['mae_mean']:.2f}€ ± {metrics['mae_std']:.2f}€")
print(f"R²:   {metrics['r2_mean']:.4f} ± {metrics['r2_std']:.4f}")
print("="*50)

# ============ ANÁLISIS DE OVERFITTING ============
print("\n Análisis de Overfitting:")
cv_results = pd.DataFrame(grid_search.cv_results_)
best_idx = grid_search.best_index_
train_score = -cv_results.loc[best_idx, 'mean_train_score']
test_score = -cv_results.loc[best_idx, 'mean_test_score']
print(f"Train RMSE (log): {train_score:.4f}")
print(f"Test RMSE (log):  {test_score:.4f}")
print(f"Diferencia:       {abs(train_score - test_score):.4f}")
if abs(train_score - test_score) < 0.05:
    print(" No hay overfitting significativo")
else:
    print("️ Posible overfitting - considera reducir complejidad del modelo")

# ============ CORRELACIONES Y DIAGNÓSTICO CON RELACIÓN A PRECIO ============
print("\n Analizando correlaciones entre features...")
X_transformed = best_pipeline.named_steps['feature_eng'].transform(X)

numeric_features = num_cols_base + num_cols_engineered
correlations = pd.DataFrame({
    'feature': numeric_features,
    'correlation_with_target': [X_transformed[col].corr(pd.Series(y)) for col in numeric_features]
}).sort_values('correlation_with_target', key=abs, ascending=False)

print("\n Correlación con log(price):")
print(correlations.to_string(index=False))

# ============ IMPORTANCIA DE FEATURES (APROXIMADA) ============
print("\n Top features por importancia (basado en correlación absoluta):")
print(correlations.head(5).to_string(index=False))

Filas iniciales: 13321
Filas después del muestreo: 1000
Filas después del filtrado: 988

 Entrenando GridSearchCV...
Fitting 5 folds for each of 100 candidates, totalling 500 fits

 Mejores parámetros: {'model__C': 5, 'model__epsilon': 0.2, 'model__gamma': 'auto'}
 Mejor score CV (neg_RMSE en log): -0.3972

 Evaluando modelo en escala original...
  Fold 1: RMSE=26.34, MAE=17.85, R²=0.5324
  Fold 2: RMSE=38.83, MAE=23.03, R²=0.3620
  Fold 3: RMSE=27.64, MAE=18.38, R²=0.5094
  Fold 4: RMSE=34.80, MAE=19.31, R²=0.3388
  Fold 5: RMSE=40.30, MAE=20.05, R²=0.3096

           RESULTADOS FINALES
RMSE: 33.58€ ± 5.69€
MAE:  19.72€ ± 1.82€
R²:   0.4105 ± 0.0920

 Análisis de Overfitting:
Train RMSE (log): 0.3601
Test RMSE (log):  0.3972
Diferencia:       0.0372
 No hay overfitting significativo

 Analizando correlaciones entre features...

 Correlación con log(price):
                       feature  correlation_with_target
             reviews_per_month                -0.097555
calculated_host_li

## 📊 Preprocesamiento y datos

1. **Filas iniciales y finales**

   * Filas iniciales: **13,321**

   * Muestra aleatoria: **1,000 registros**

   * Tras filtrado de outliers: **988 registros**

   > Se mantiene un tamaño de muestra representativo. El filtrado de outliers mediante los percentiles 5.º y 95.º elimina valores extremos del precio, lo que reduce el ruido y evita que el modelo se vea afectado por precios anómalos. Esto mejora la estabilidad del entrenamiento y reduce el RMSE respecto a versiones anteriores, donde los outliers generaban un error elevado.

2. **Tratamiento de variables**

   * Eliminación de `neighbourhood` (alta cardinalidad).

   * Mantenimiento de `neighbourhood_group` como variable geográfica simplificada.

   * Reemplazo de ceros por *NaN* en variables con posible ausencia real de valor (`number_of_reviews`, `reviews_per_month`, `availability_365`).

   * Limitación de `minimum_nights` a un máximo de 365 días.

   > Estas transformaciones mejoraron la coherencia de los datos y redujeron ruido. En versiones previas, las imputaciones incorrectas y la presencia de categorías redundantes causaban una menor capacidad de generalización y un sobreajuste leve en los conjuntos de validación.

3. **Ingeniería de características**

   * Creación de `distance_to_center` a partir de coordenadas y eliminación de `latitude` y `longitude`.

   > Esta nueva variable aporta una medida interpretable y compacta de ubicación, reduciendo la dimensionalidad y mejorando la estabilidad del modelo.
   > En resultados previos sin esta feature, el modelo mostraba un **R² ≈ 0.32** y **RMSE ≈ 39 €**, mientras que con esta transformación se alcanzó **R² ≈ 0.41** y **RMSE ≈ 33.6 €**, reflejando una mejora real de la capacidad predictiva gracias a esta incorporación.

4. **Transformación del target**

   * Aplicación de `log1p(price)` para estabilizar la varianza.

   * Las métricas finales se reportan en la escala original mediante `expm1`.

   > En implementaciones anteriores, sin transformación logarítmica, el modelo tendía a sobreestimar precios altos y subestimar precios bajos, generando una dispersión mayor del error. Con esta transformación, la distribución del target se volvió más simétrica, mejorando notablemente la precisión global.

---

## ⚙️ Pipeline y modelo

El flujo se implementó en un **pipeline reproducible y sin fugas de información**, compuesto por:

* **Preprocesamiento:** imputación, escalado estándar y codificación *OneHot*.
* **FeatureEngineer personalizado:** añade `distance_to_center`.
* **Regresor SVR (kernel RBF):** modelo base no lineal.

> La integración de todos los pasos dentro del pipeline garantizó la coherencia entre entrenamiento y validación, evitando contaminación de datos.
> En versiones anteriores, el preprocesamiento externo al pipeline generaba ligeras inconsistencias entre folds, reflejadas en una varianza mayor de las métricas.

---

## 🔎 Validación y tuning

* **Validación cruzada:** `KFold(n_splits=5, shuffle=True, random_state=0)`
* **Métrica principal:** RMSE (negativo en búsqueda)
* **Optimización de hiperparámetros:** búsqueda en cuadrícula (`GridSearchCV`) sobre `C`, `gamma` y `epsilon`.

### 🔧 Mejor configuración obtenida

```python
{'model__C': 5, 'model__epsilon': 0.2, 'model__gamma': 'auto'}
```

> En versiones anteriores, se usaban valores predeterminados (`C=1`, `gamma='scale'`, `epsilon=0.1`), que generaban menor flexibilidad del modelo.
> El nuevo ajuste logró un balance adecuado entre sesgo y varianza, reduciendo el error sin sobreajuste.

---

## 📈 Resultados y evaluación

| Métrica  | Media (CV 5 folds) | Desviación |
| :------- | -----------------: | ---------: |
| **RMSE** |            33.58 € |   ± 5.69 € |
| **MAE**  |            19.72 € |   ± 1.82 € |
| **R²**   |             0.4105 |   ± 0.0920 |

> Comparativamente, las métricas previas del mismo modelo antes de la optimización eran:
> **RMSE ≈ 39.0 €, MAE ≈ 23.1 €, R² ≈ 0.32**, lo que evidencia una **reducción del error absoluto medio en más de 3 €** y un **aumento del poder explicativo del modelo del 32 % al 41 %**.

---

## 🧠 Análisis de resultados

1. **Desempeño general:**
   El modelo mejorado captura una proporción mayor de la variabilidad del precio gracias a un flujo de preprocesamiento más consistente y al uso de una variable geográfica representativa. Los errores absolutos se han reducido de forma significativa.

2. **Regularización y generalización:**

   * RMSE (train log): 0.3601

   * RMSE (test log): 0.3972

   * Diferencia: 0.0371

   > La diferencia es pequeña, indicando **buena capacidad de generalización**. En versiones previas, la diferencia superaba los 0.08 puntos log, señal de sobreajuste moderado.

3. **Correlaciones con el precio (log):**

| Feature                          | Correlación |
| :------------------------------- | ----------: |
| `reviews_per_month`              |      -0.098 |
| `calculated_host_listings_count` |      -0.078 |
| `availability_365`               |      -0.065 |
| `distance_to_center`             |      -0.059 |
| `minimum_nights`                 |      +0.022 |

> Las correlaciones directas siguen siendo bajas, lo que refuerza la elección del **kernel RBF** para modelar relaciones no lineales.
> Sin embargo, tras la ingeniería de características y limpieza, estas correlaciones son más estables y coherentes con la lógica del dominio (mayor distancia → menor precio).

---

## 🔍 Conclusión

El modelo final de **regresión SVR (RBF)**, combinado con un preprocesamiento robusto, la eliminación de outliers, y la introducción de `distance_to_center`, representa una mejora sustancial respecto a las versiones anteriores.
El nuevo flujo ha permitido:

* Reducir **RMSE** de ~39 € a **33.6 €**
* Mejorar **R²** de **0.32 → 0.41**
* Mantener una **generalización estable** sin sobreajuste
* Obtener métricas más consistentes entre folds

En conjunto, los resultados muestran que las mejoras en el preprocesamiento, la ingeniería de características y la optimización de parámetros han **aumentado significativamente la precisión y estabilidad del modelo**, demostrando un flujo de trabajo más maduro y controlado.